In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

- red wine 품질 등급 예측
```
1. 데이터셋 확보 & 전처리
    독립변수와 타켓변수 분리 -> 독립변수 스케일조정(StendardScaler) 
    -> 타겟변수원핫인코딩(get_dummies) -> 훈련셋과 테스트셋분리(train_test_split)
2. 모델 구성(입력11, 출력?)
3. 모델학습과정 설정 (다중분류로 설정)
```

In [4]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import matplotlib.pyplot as plt

# 1. 데이터셋 확보 & 전처리
    독립변수와 타켓변수 분리 -> 독립변수 스케일조정(StendardScaler) 
    -> 타겟변수원핫인코딩(get_dummies) -> 훈련셋과 테스트셋분리(train_test_split)

In [9]:
# np.loadtxt('data/winequality-red.csv',delimiter=';', skiprows=1)
# np.genfromtxt('data/winequality-red.csv',delimiter=';', skip_header=1)
redwine = pd.read_csv('data/winequality-red.csv',sep=';')
# 고정 산도(fixed acidity)
# 휘발성 산도(volatile acidity)
# 구연산(citric acid)
# 잔여 당분(residual sugar)
# 염화물(chlorides)
# 유리 아황산염(free sulfur dioxide)
# 총 아황산염(total sulfur dioxide)
# 밀도(density)
# pH
# 황산염(sulphates)
# 알코올(alcohol)
redwine.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [11]:
redwine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [13]:
# 독립변수, 타겟변수 분리
X_redwine = redwine.iloc[:, :-1].values
y_redwine = redwine.iloc[:, -1]
X_redwine.shape, y_redwine.shape

((1599, 11), (1599,))

In [16]:
# 독립변수 X의 스케일 조정
scaler = StandardScaler()
scaler.fit(X_redwine)
scaled_X_redwine = scaler.transform(X_redwine)
# 다중분류를 위한 타겟변수의 원핫인코딩 -> numpy배열로 변환
Y_redwine = pd.get_dummies(y_redwine).to_numpy()


In [17]:
# 독립변수 & 타겟변수
scaled_X_redwine.shape, Y_redwine.shape

((1599, 11), (1599, 6))